In [80]:
import os
import sys
from pathlib import Path

# lägg till projektroten (mappen ovanför notebooks/) på sys.path
root_dir = Path().absolute()

if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])

root_dir = str(root_dir) 
print(f"Root dir: {root_dir}")
print("Local environment")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Root dir: c:\Users\Chris\hockey-agent
Local environment


In [81]:
import hopsworks
from config import settings
import requests
import pandas as pd

In [82]:
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY,
    host = settings.HOPSWORKS_HOST
)


2025-12-15 13:02:44,342 INFO: Closing external client and cleaning up certificates.
2025-12-15 13:02:44,348 INFO: Connection closed.
2025-12-15 13:02:44,350 INFO: Initializing external client
2025-12-15 13:02:44,351 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-15 13:02:45,569 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3193


In [83]:
fs = project.get_feature_store()

fg = fs.get_feature_group(
    name="player_season_stats",
    version=1
)

df = fg.read()
df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.86s) 


,assists,ev_goals,ev_points,faceoff_win_pct,game_winning_goals,games_played,goals,ot_goals,penalty_minutes,player_id,...,pp_goals,pp_points,sh_goals,sh_points,shooting_pct,shoots_catches,shots,skater_full_name,time_on_ice_per_game,season_id
0,20,17,27,0.32941,0,82,24,0,33,8480188,...,6,16,1,1,0.11764,R,204,Fabian Zetterlund,1131.3658,20232024
1,34,8,32,NaN,2,80,13,0,24,8478396,...,5,13,0,2,0.07926,L,164,Noah Hanifin,1416.8375,20232024
2,17,1,16,NaN,0,78,1,0,43,8480049,...,0,1,0,1,0.01298,L,77,Dylan Samberg,937.5384,20232024
3,15,1,15,NaN,1,58,1,1,27,8476917,...,0,0,0,1,0.01408,L,71,Adam Pelech,1216.8965,20232024
4,0,1,1,0.30769,0,2,1,0,0,8475413,...,0,0,0,0,0.33333,L,3,Justin Dowling,631.5000,20232024


In [84]:
def top_players(
    df,
    position=None,
    metric="points",
    n=10
):
    data = df.copy()

    if position == "F":
        data = data[data["position_code"].isin(["C", "L", "R"])]
    elif position:
        data = data[data["position_code"] == position]

    data = data[data[metric].notna()]

    return (
        data.sort_values(metric, ascending=False)
            .head(n)
            [["skater_full_name", "position_code", metric, "games_played"]]
    )


In [85]:
question = "Vilka forwards gjorde flest poäng?"
#print(df.columns.tolist())

result = top_players(
    df,
    position="F",
    metric="points",
    n=5
)

result


,skater_full_name,position_code,points,games_played
714,Nikita Kucherov,R,144,81
150,Nathan MacKinnon,C,140,82
905,Connor McDavid,C,132,76
776,Artemi Panarin,L,120,82
356,David Pastrnak,R,110,82


In [86]:
import os
from dotenv import load_dotenv
load_dotenv()

import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-flash-lite")

In [ ]:
# Here we can add more tools 
TOOLS_DESCRIPTION = """
You can use the following tools:

1. top_players:
   Use when the user asks for best players, rankings, leaders.
   Parameters:
   - position: "F", "D", "C", "L", "R" or null
   - metric: one of ["points", "points_per_game", "ev_points", "goals"]
   - n: number of players

Return ONLY valid JSON.
"""


In [88]:
def decide_tool(question: str):
    prompt = f"""
You are a hockey data assistant.

{TOOLS_DESCRIPTION}

User question:
"{question}"

Decide which tool to use and return JSON like:
{{
  "tool": "top_players",
  "position": "F",
  "metric": "points",
  "n": 5
}}
"""

    response = model.generate_content(prompt)
    return response.text


In [89]:
decision = decide_tool("Vilka forwards gjorde flest poäng?")
print(decision)


```json
{
  "tool": "top_players",
  "position": "F",
  "metric": "points",
  "n": 5
}
```


In [90]:
import json

def run_agent(question: str, df):
    decision = decide_tool(question)
    params = json.loads(decision)

    if params["tool"] == "top_players":
        result = top_players(
            df,
            position=params.get("position"),
            metric=params.get("metric"),
            n=params.get("n", 5)
        )
        return result
run_agent("Vilka forwards var bäst i 5 mot 5?", df)


,skater_full_name,position_code,ev_points,games_played
150,Nathan MacKinnon,C,92,82
714,Nikita Kucherov,R,91,81
905,Connor McDavid,C,87,76
824,Auston Matthews,C,77,81
356,David Pastrnak,R,75,82


In [93]:
def explain_result(question, table_text):
    prompt = f"""
User question:
{question}

Data:
{table_text}

Explain the result in clear hockey terms in Swedish.
"""
    response = model.generate_content(prompt)
    return response.text

# Run the agent
result = run_agent("Vilka forwards gjorde flest poäng?", df)
table_text = result.to_string(index=False, justify="left")
explanation = explain_result("Vilka forwards gjorde flest poäng?", table_text)
print(explanation)

Resultatet visar att **Nikita Kucherov** gjorde flest poäng med 144 poäng.

I hockeytermer betyder detta att Kucherov var den spelare i den här gruppen som totalt sett var mest produktiv under säsongen. Poäng i ishockey räknas som summan av gjorda mål och assists. En spelare som Kucherov med 144 poäng har alltså antingen gjort väldigt många mål själv, eller spelat fram sina lagkamrater till mål väldigt många gånger, eller en kombination av båda.

För att sätta det i perspektiv:

*   **Nikita Kucherov** (144 poäng) är den absolut bästa poänggöraren i den här listan.
*   **Nathan MacKinnon** (140 poäng) och **Connor McDavid** (132 poäng) följer strax efter och är också bland de allra bästa offensiva spelarna.
*   **Artemi Panarin** (120 poäng) och **David Pastrnak** (110 poäng) är också exceptionellt bra poängplockare, men ligger något bakom toppen.

Att Kucherov toppar listan, trots att han spelat lika många matcher som flera andra spelare (81 matcher), indikerar att han hade en otrolig